In [6]:
#lilo
%cd '/content/drive/MyDrive/Colab Notebooks/preferential-attachment/notebooks'

/content/drive/MyDrive/Colab Notebooks/preferential-attachment/notebooks


In [8]:
# !git config user.email yossi.cohen@live.com
# !git config user.name "Yossi Cohen"
!git commit -am "poisson-exp-3" && git push

[master 36c30e1] poisson-exp-3
 2 files changed, 2 insertions(+), 1 deletion(-)
 rewrite notebooks/poisson-exp-2.ipynb (89%)
 create mode 100644 notebooks/poisson-exp-3.ipynb
Counting objects: 5, done.
Delta compression using up to 2 threads.
Compressing objects: 100% (5/5), done.
Writing objects: 100% (5/5), 1.88 KiB | 642.00 KiB/s, done.
Total 5 (delta 2), reused 0 (delta 0)
remote: Resolving deltas: 100% (2/2), completed with 2 local objects.
To https://github.com/yossi-cohen/preferential-attachment
   ed26564..36c30e1  master -> master
